# [LVV-T1876] - Settling Time After a Slew

Notebook containing data analysis for the [LVV-T1876] test case.  
~The script used to run this test case can be found in [lsst-ts/ts_m1m3supporttesting/M13T012.py].~


[LVV-T1876]: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/2237
[lsst-ts/ts_m1m3supporttesting/M13T012.py]: https://github.com/lsst-ts/ts_m1m3supporttesting/blob/develop/M13T012.py


see [LVV-T2732_analysis_mt_encoder_slew_jitter.ipynb] for jitter ideas?

[LVV-T2732_analysis_mt_encoder_slew_jitter.ipynb]: https://github.com/lsst-sitcom/notebooks_vandv/blob/develop/notebooks/tel_and_site/subsys_req_ver/tma/LVV-T2732_analysis_mt_encoder_slew_jitter.ipynb


My first thought is we:
1. identify appropriate slew events (?)
2.  Compute rms for time window after 'inPosition' log event
3. Define settled when all rms are below some value (not sure what the threshold should be)


Worked on

## Prepare Notebook

In [ ]:
test_case = "LVV-T235"
test_exec = "LVV-E985"

t_start = "2023-03-07T18:48:20" # Exact time clean up initial fault
t_end = "2023-03-07T19:30:00"

delta_t = 2 # seconds

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import itertools as itt
import pandas as pd
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from astropy import units as u
from astropy.time import Time, TimezoneInfo

from lsst.sitcom import vandv
from lsst.ts.idl.enums import MTM1M3

In [ ]:
delta_t = pd.Timedelta(delta_t, "seconds")
delta_z = [pd.Timedelta(1, "seconds"),pd.Timedelta(10, "seconds")]
client = vandv.efd.create_efd_client()

In [ ]:
## Query Data

In [ ]:
all_columns = ["xPosition", "xRotation", "yPosition", "yRotation", "zPosition", "zRotation"]
pos_columns = [c for c in all_columns if "Position" in c]
rot_columns = [c for c in all_columns if "Rotation" in c]
print(pos_columns, rot_columns)

In [ ]:
df_ims = await client.select_time_series(
    "lsst.sal.MTM1M3.imsData", 
    "*", 
    Time(t_start, format="isot", scale="utc"),
    Time(t_end, format="isot", scale="utc"), 
)

df_ims = df_ims.set_index("private_rcvStamp")
df_ims.index = pd.to_datetime(df_ims.index, unit="s")
df_ims = df_ims[all_columns]

# Convert meter to milimeter to make is easier to analyse
df_ims[pos_columns] = df_ims[pos_columns] * 1e3

# Convert radians to degrees
df_ims[rot_columns] = np.rad2deg(df_ims[rot_columns])

In [ ]:
df_state = await client.select_time_series(
    "lsst.sal.MTM1M3.logevent_detailedState", 
    "*", 
    Time(t_start, format="isot", scale="utc"),
    Time(t_end, format="isot", scale="utc"), 
)

df_state["detailedStateName"] = \
    df_state["detailedState"].map(lambda x: MTM1M3.DetailedState(x).name)

df_state = df_state.set_index("private_rcvStamp")
df_state.index = pd.to_datetime(df_state.index, unit="s")

In [ ]:
df_cmdPos = await client.select_time_series(
    "lsst.sal.MTM1M3.command_positionM1M3", 
    "*", 
    Time(t_start, format="isot", scale="utc"),
    Time(t_end, format="isot", scale="utc"), 
)

df_cmdPos = df_cmdPos.set_index("private_rcvStamp")
df_cmdPos.index = pd.to_datetime(df_cmdPos.index, unit="s")
df_cmdPos = df_cmdPos[all_columns]
df_cmdPos = df_cmdPos * 1e3 # Convert meter to milimeter to make is easier to analyse

In [ ]:
df_HPState = await client.select_time_series(
    "lsst.sal.MTM1M3.logevent_hardpointActuatorState", 
    "*", 
    Time(t_start, format="isot", scale="utc"),
    Time(t_end, format="isot", scale="utc"), 
)

df_HPState = df_HPState.set_index("private_rcvStamp")
df_HPState.index = pd.to_datetime(df_HPState.index, unit="s")

## Initial Data Display 

In [ ]:
title = f"{test_case} {test_exec}\nData Overview"
fig, axs = plt.subplots(num=title, nrows=3, sharex=True, figsize=(10, 5))

when_parked = df_state[df_state["detailedStateName"] == "PARKED"].index
when_rasing = df_state[df_state["detailedStateName"] == "RAISINGENGINEERING"].index
when_lowering = df_state[df_state["detailedStateName"] == "LOWERINGENGINEERING"].index
when_active = df_state[df_state["detailedStateName"] == "ACTIVEENGINEERING"].index

for i, label in enumerate(pos_columns):
    ax = axs[i]
    ax.plot(df_ims[label])
    
    for idx in when_parked:
        l1 = ax.axvline(idx, lw="0.5", c="k")
        
    for idx in when_rasing:
        l2 = ax.axvline(idx, lw="0.5", c="k", ls="--")
        
    for idx in when_lowering:
        l3 = ax.axvline(idx, lw="0.5", c="k", ls=":")
        
    for idx in when_active:
        l4 = ax.axvline(idx, lw="0.5", c="C1", ls="-")

    ax.grid(":", lw=0.1)
    ax.set_xlabel("Time [UTC]")
    ax.set_ylabel(f"{label} [mm]")

fig.legend(
    [l1, l2, l3, l4], 
    ["PARKED", "RAISINGENGINEERING", "LOWERINGENGINEERING", "ACTIVEENGINEERING"], 
    ncols=4, 
    loc="upper right", 
    bbox_to_anchor=(0.93, 0.92)
)
fig.suptitle(title + "\n")
fig.autofmt_xdate()
fig.tight_layout()

plt.show()

In [ ]:
#alternative representation
df_ims['xPosition'].plot(color='red', label='xPosition (mm)')
df_ims['yPosition'].plot(color='blue', label='yPosition (mm)')
df_ims['zPosition'].plot(color='green', label='zPosition (mm)')
df_ims['xRotation'].plot(color='red', linestyle='dashed', label='xRotation (degrees)')
df_ims['yRotation'].plot(color='blue', linestyle='dashed', label='yRotation (degrees)')
df_ims['zRotation'].plot(color='green', linestyle='dashed', label='zRotation (degrees)')
plt.legend(ncols=2, loc="upper right", bbox_to_anchor=(0.80, 1.3))
plt.xlabel("Time [UTC]")
fig.autofmt_xdate()
fig.tight_layout()

In [ ]:
#Zoomed in version with T0 and delta_t
T0 = '2023-03-07T19:00:00'
T0 = pd.to_datetime(T0)
T1 = T0 + delta_t
print(T0,T1)

In [ ]:
fig = plt.figure()
TZoom = [T0-delta_z[0], T0+delta_z[1]] 
df_ims['xPosition'][TZoom[0]:TZoom[1]].plot(color='red', label='xPosition (mm)')
df_ims['yPosition'][TZoom[0]:TZoom[1]].plot(color='blue', label='yPosition (mm)')
df_ims['zPosition'][TZoom[0]:TZoom[1]].plot(color='green', label='zPosition (mm)')
df_ims['xRotation'][TZoom[0]:TZoom[1]].plot(color='red', linestyle='dashed', label='xRotation (degrees)')
df_ims['yRotation'][TZoom[0]:TZoom[1]].plot(color='blue', linestyle='dashed', label='yRotation (degrees)')
df_ims['zRotation'][TZoom[0]:TZoom[1]].plot(color='green', linestyle='dashed', label='zRotation (degrees)')
plt.legend(ncols=2, loc="upper right", bbox_to_anchor=(0.80, 1.3))
plt.axvline(T0, lw="2", c="k")
plt.axvline(T1, lw="0.5", c="k")
plt.axvspan(T0, T1, alpha=0.5, color='red')
plt.xlabel("Time [UTC]")
fig.autofmt_xdate()
fig.tight_layout()

In [ ]:
when_active = df_state[df_state["detailedStateName"] == "ACTIVEENGINEERING"].index

sub_df = pd.DataFrame(columns=df_ims.columns.to_list())

for idx in when_active:
    dt = pd.Timedelta(2, "sec")
    temp = df_ims.loc[idx:idx+dt]
    sub_df = pd.concat((sub_df, temp), axis=0)
    
median_vals = sub_df.median()
print(median_vals)
df_results = df_ims.copy() 

for col in pos_columns:
    df_results[col] = df_results[col] - median_vals[col]

In [ ]:
title = f"{test_case} {test_exec}\nData Overview with Reference Subtracted"
fig, axs = plt.subplots(num=title, nrows=3, sharex=True, figsize=(10, 5))

when_parked = df_state[df_state["detailedStateName"] == "PARKED"].index

for i, label in enumerate(["xPosition", "yPosition", "zPosition"]):
    ax = axs[i]
    ax.plot(df_results[label])
    
    for idx in when_parked:
        ax.axvline(idx, lw="0.5", c="k")
        
    for idx in when_rasing:
        ax.axvline(idx, lw="0.5", c="k", ls="--")
        
    for idx in when_lowering:
        ax.axvline(idx, lw="0.5", c="k", ls=":")
        
    for idx in when_active:
        ax.axvline(idx, lw="0.5", c="C1", ls="-")

    ax.grid(":", lw=0.1)
    ax.set_xlabel("Time [UTC]")
    ax.set_ylabel(f"{label} [mm]")

fig.legend(
    [l1, l2, l3, l4], 
    ["PARKED", "RAISINGENGINEERING", "LOWERINGENGINEERING", "ACTIVEENGINEERING"], 
    ncols=4, 
    loc="upper right", 
    bbox_to_anchor=(0.93, 0.92)
)
    
fig.suptitle(title + "\n")
fig.autofmt_xdate()
fig.tight_layout()

plt.show()

In [ ]:
from scipy.interpolate import UnivariateSpline
def get_vel(times, positions, interp_times):
    position_spline=UnivariateSpline(times, positions, s=1)
    velocity_spline=position_spline.derivative(n=1)
    return velocity_spline(interp_times)
    

In [ ]:
df_results["time"]=(df_results.index.values-df_results.index.values[0]) / np.timedelta64(1, 's') 

In [ ]:
df_results["time"].max()

In [ ]:
interp_times=np.linspace(0,df_results["time"].max(),4000)
vel=get_vel(df_results["time"], df_results["xPosition"], df_results["time"])

In [ ]:

plt.figure(dpi=175)
#plt.plot(df_results["time"], df_results["xPosition"]*vel.max()/df_results["xPosition"].max() )
# plt.plot(df_results["time"], vel)
# plt.plot(df_results["time"],df_results["xPosition"].rolling(3).std())
sel=(vel < 0.01)
plt.scatter(df_results["time"][sel],df_results["xPosition"].rolling(3).std()[sel])
plt.scatter(df_results["time"][~sel],df_results["xPosition"].rolling(3).std()[~sel])


In [ ]:
fig, axs = plt.subplots(num=title, nrows=3, sharex=True, figsize=(10, 5))
when_parked = df_state[df_state["detailedStateName"] == "PARKED"].index

for i, label in enumerate(["xPosition", "yPosition", "zPosition"]):
    ax = axs[i]
    ax.plot(df_results[label])
    
    for idx in when_parked:
        ax.axvline(idx, lw="0.5", c="k")
        
    for idx in when_rasing:
        ax.axvline(idx, lw="0.5", c="k", ls="--")
        
    for idx in when_lowering:
        ax.axvline(idx, lw="0.5", c="k", ls=":")
        
    for idx in when_active:
        ax.axvline(idx, lw="0.5", c="C1", ls="-")

    ax.grid(":", lw=0.1)
    ax.set_xlabel("Time [UTC]")
    ax.set_ylabel(f"{label} [mm]")
fig.suptitle(title + "\n")
fig.autofmt_xdate()
fig.tight_layout()

plt.show()